In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re
import json
import pandas as pd
from datetime import date
from tqdm.auto import tqdm
import ipywidgets

In [2]:
today = str(date.today())

In [3]:
dummy_url = "http://memory.pvost.org/pages/dic.html"
page = urllib.request.urlopen(dummy_url)

In [4]:
#ищем все линки
soup = BeautifulSoup(page)
links = []
for item in soup("li", {"class": "main"}):
    for link in item.find_all("a"):
        if "href" in link.attrs:
            links.append(link["href"])

In [5]:
#подготовленные имена для определения гендера
ambig_fem = ["Любовь", "Рашель", "Эсфирь", "Элен", "Нуне", "Исоко", "Нинель", "Мариам"]
ambig_male = ["Абдулла", "Илья", "Дзига", "Никита", "Махмуд-ходжа", "Шевки", "Бекги", "Текки", "Шалва", "Вали", "Мамия", "Эльбек-Доржи", "Абони", "Грачия", "Рухулла", "Мяти", "Цви", "Чайжи-Дава", "Хаджи", "Габдел-Бари", "Ахмет-Заки", "Бидия", "Гази-Али", "Даши", "Хади", "Ханафи", "Баруди", "Али", "Хужа", "Шота", "Хамди", "Абибулла", "Вели", "Ягья", "Мухаметша", "Ахмедхади", "Амди", "Габдулла", "Ушана", "Эрдени", "Осман Нури"]

In [61]:
#собираем словарь с данными
biographies = []
for link in tqdm(links): 
    page = "http://memory.pvost.org/pages/" + link
    try:
        page = urllib.request.urlopen(page)
    except:
        continue
    soup_bio = BeautifulSoup(page)
    east_bio = {}
    info = soup_bio.find("p", {"class": "zag2"})
    names = info.text
    names = re.sub(r'\([^()]*\)', "", names)
    names = re.sub(r'\([^()]*\)', "", names)
    try:
        dates = re.search(r"((\d{4}\??|\?)-?(\d{4}\??|\?)?)", info.text).group()
    except:
        continue
    try:
        surname = re.search(r"[А-ЯЁ]+-?[А-ЯЁ]+", names).group()
        east_bio["surname"] = surname.title()
    except:
        east_bio["surname"] = ""
    try: 
        birth = re.search(r"(\d{4}\??|\?)", dates).group()
    except:
        continue
    try:
        firstname = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]{0,10})|([А-ЯЁ]\.)", names).group()
        east_bio["firstname"] = firstname
    except:
        firstname = ""
        east_bio["firstname"] = firstname
    #try:
    #    firstname = re.search(r"([А-Я]{1}[а-я]+-?[А-Яа-я]+)", names).group()
    #except AttributeError:
    #    continue
    try:
        patronym = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]+)|([А-ЯЁ]\.)", names.split(firstname+" ")[1]).group()
        east_bio["patronym"] = patronym
        east_bio["fullname"] = surname.title()+" "+firstname+" "+patronym
    except:
        patronym = ""
        east_bio["patronym"] = patronym
        east_bio["fullname"] = surname.title()+" "+firstname
    if re.search("(а|я|и)$", firstname) and firstname not in ambig_male:
        east_bio["gender"] = "женщина"
    elif firstname in ambig_fem:
        east_bio["gender"] = "женщина"
    else:
        east_bio["gender"] = "мужчина"
    #try:
    #    east_bio["fullname"] = surname.title()+" "+firstname+" "+patronym
    #except NameError:
    #    east_bio["fullname"] = surname.title()+" "+firstname
    try:
        east_bio["birthdate"] = birth
    except:
        east_bio["birthdate"] = ""
    try: 
        biography = soup_bio("p", attrs={'class': None})
        for i in range(len(biography)):
            birth_check = re.search(r"[а-яё]+", biography[0].text)
            if birth_check:
                biography_info = re.sub(r"\n\s{2,30}", "", biography[0].text)
                birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography_info.split("Род. в ")[1]).group()
                if birthplace:
                    east_bio["birthplace"] = birthplace
            else:
                birth_check = re.search(r"[а-яё]+", biography[1].text)
                if birth_check:
                    biography_info = re.sub(r"\n\s{2,30}", "", biography[1].text)
                    birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography_info.split("Род. в ")[1]).group()
                    if birthplace:
                        east_bio["birthplace"] = birthplace
                else:
                    biography_info = re.sub(r"\n\s{2,30}", "", biography[2].text)
                    birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography_info.split("Род. в ")[1]).group()
                    if birthplace:
                        east_bio["birthplace"] = birthplace
    except:
        east_bio["birthplace"] = ""
    try:
        east_bio["death_date"] = re.search(r"1[8-9][0-9]{2}", dates.split(birth)[1]).group()
    except:
        east_bio["death_date"] = ""
    east_bio["original_link"] = "http://memory.pvost.org/pages/" + link
    east_bio["retrieved_at"] = today
    biographies.append(east_bio)

  0%|          | 0/771 [00:00<?, ?it/s]

In [62]:
#смотрим, что получилось в датафрейме
bio_parse = pd.DataFrame(biographies)

In [63]:
bio_parse

,surname,firstname,patronym,fullname,gender,birthdate,birthplace,death_date,original_link,retrieved_at
0,Абих,Рудольф,Петрович,Абих Рудольф Петрович,мужчина,1901,г. Баку,1940,http://memory.pvost.org/pages/abrih.html,2021-05-18
1,Абрамский,Иехезкель,,Абрамский Иехезкель,мужчина,1886,м. Дашковичи Гродненской губ,1976,http://memory.pvost.org/pages/abramskij.html,2021-05-18
2,Абрамсон,Мануил,Моисеевич,Абрамсон Мануил Моисеевич,мужчина,1898,др. свед,1938,http://memory.pvost.org/pages/abramson.html,2021-05-18
3,Абугов,Михаил,Захарович,Абугов Михаил Захарович,мужчина,1901,зам. зав,1941,http://memory.pvost.org/pages/abugov.html,2021-05-18
4,Абузов,Степан,Семенович,Абузов Степан Семенович,мужчина,1898,с. Старые Бегучи Самарской губ,1938,http://memory.pvost.org/pages/abuzov.html,2021-05-18
...,...,...,...,...,...,...,...,...,...,...
737,Чернышев,Евгений,Иванович,Чернышев Евгений Иванович,мужчина,1894,г. Симбирск,,http://memory.pvost.org/pages/ds_chernishev.html,2021-05-18
738,Шийк,Андрей,Александрович,Шийк Андрей Александрович,мужчина,1891,,1978,http://memory.pvost.org/pages/ds_shijk.html,2021-05-18
739,Элиасов,Лазарь,Ефимович,Элиасов Лазарь Ефимович,мужчина,?,,,http://memory.pvost.org/pages/ds_elisarov.html,2021-05-18
740,Янчевецкий,Дмитрий,Григорьевич,Янчевецкий Дмитрий Григорьевич,мужчина,1889,,1934,http://memory.pvost.org/pages/ds_yanchevitskij...,2021-05-18


In [43]:
names = set()
for item in bio_parse["firstname"]:
    names.add(item)

In [44]:
female = []
male = []
ambig_fem = ["Любовь", "Рашель", "Эсфирь", "Элен", "Нуне", "Исоко", "Нинель", "Мариам"]
ambig_male = ["Абдулла", "Илья", "Дзига", "Никита", "Махмуд-ходжа", "Шевки", "Бекги", "Текки", "Шалва", "Вали", "Мамия", "Эльбек-Доржи", "Абони", "Грачия", "Рухулла", "Мяти", "Цви", "Чайжи-Дава", "Хаджи", "Габдел-Бари", "Ахмет-Заки", "Бидия", "Гази-Али", "Даши", "Хади", "Ханафи", "Баруди", "Али", "Хужа", "Шота", "Хамди", "Абибулла", "Вели", "Ягья", "Мухаметша", "Ахмедхади", "Амди", "Габдулла", "Ушана", "Эрдени"] 
for name in names:
    if re.search("(а|я|и)$", name) and name not in ambig_male:
        female.append(name)
    elif name in ambig_fem:
        female.append(name)
    else:
         male.append(name)

In [45]:
male

['',
 'Карим',
 'Ушана',
 'Хаджи',
 'Самсон',
 'Дзахо',
 'Макар',
 'Эйзер',
 'Мирсаид',
 'Абульфаз',
 'Гурий',
 'Венедикт',
 'Вукол',
 'Петр',
 'Леонид',
 'Арсений',
 'Гирсиль',
 'Глеб',
 'Карл',
 'Осор',
 'Худайберген',
 'Фердинанд',
 'Цви',
 'Шерван',
 'Лев',
 'Абдулла',
 'Нигмат',
 'Вильгельм',
 'Африкан',
 'Марат',
 'Мстислав',
 'Вели',
 'Алибек',
 'Василий',
 'Жумахан',
 'Салах',
 'Бекги',
 'Базаржаб',
 'Всеволод',
 'Эльдес',
 'Зотик',
 'Араб',
 'Хан',
 'Асан',
 'Дамдин',
 'Халид',
 'Артур',
 'Галимджан',
 'Лаврентий',
 'Нурэддин',
 'Цыбен',
 'Абибулла',
 'Андрей',
 'Микаэл',
 'Хасан',
 'Мосэ',
 'Исмаил',
 'Данзан',
 'Султан-Меджид',
 'Никита',
 'Лазарь',
 'Габдулла',
 'Хазиме',
 'Хамди',
 'Эрдени',
 'Эрвин',
 'Пантелеймон',
 'Моисей',
 'Балжир',
 'Теодор',
 'Георгий',
 'Антанас',
 'Альберт',
 'Мин-У',
 'Иехиель',
 'Аркадий',
 'Мидхат',
 'Диодор',
 'Страшемир',
 'Чайжи-Дава',
 'Абид',
 'Илья',
 'Вали',
 'Гайк',
 'Наум',
 'Габбас',
 'Валериан',
 'Мир-Якуб',
 'Стахей',
 'Санджар',
 

In [46]:
female_regex = re.compile(""+"|".join(female))
#regex = re.compile("^("+"|".join(item_list)+")\.")
male_regex = re.compile(""+"|".join(male))

In [47]:
gender = []
for item in bio_parse["firstname"]:
    fem_match = re.search(female_regex, str(item))
    male_match = re.search(male_regex, str(item))
    if fem_match:
        gender.append("женщина")
    elif male_match:
        gender.append("мужчина")
    else:
        gender.append("")
bio_parse["gender"] = gender

In [48]:
bio_parse

,surname,firstname,patronym,fullname,gender,birthdate,birthplace,death_date,original_link,retrieved_at
0,Абих,Рудольф,Петрович,Абих Рудольф Петрович,мужчина,1901,г. Баку,1940,http://memory.pvost.org/pages/abrih.html,2021-05-18
1,Абрамский,Иехезкель,,Абрамский Иехезкель,мужчина,1886,м. Дашковичи Гродненской губ,1976,http://memory.pvost.org/pages/abramskij.html,2021-05-18
2,Абрамсон,Мануил,Моисеевич,Абрамсон Мануил Моисеевич,мужчина,1898,др. свед,1938,http://memory.pvost.org/pages/abramson.html,2021-05-18
3,Абугов,Михаил,Захарович,Абугов Михаил Захарович,мужчина,1901,зам. зав,1941,http://memory.pvost.org/pages/abugov.html,2021-05-18
4,Абузов,Степан,Семенович,Абузов Степан Семенович,мужчина,1898,с. Старые \n Бегучи Самар...,1938,http://memory.pvost.org/pages/abuzov.html,2021-05-18
...,...,...,...,...,...,...,...,...,...,...
737,Чернышев,Евгений,Иванович,Чернышев Евгений Иванович,мужчина,1894,г. Симбирск,,http://memory.pvost.org/pages/ds_chernishev.html,2021-05-18
738,Шийк,Андрей,Александрович,Шийк Андрей Александрович,мужчина,1891,,1978,http://memory.pvost.org/pages/ds_shijk.html,2021-05-18
739,Элиасов,Лазарь,Ефимович,Элиасов Лазарь Ефимович,мужчина,?,,,http://memory.pvost.org/pages/ds_elisarov.html,2021-05-18
740,Янчевецкий,Дмитрий,Григорьевич,Янчевецкий Дмитрий Григорьевич,мужчина,1889,,1934,http://memory.pvost.org/pages/ds_yanchevitskij...,2021-05-18


In [59]:
#тестовый парсинг одной статьи
#dummy_url_again = "http://memory.pvost.org/pages/avdoschenkov.htm"
#dummy_url_again = "http://memory.pvost.org/pages/abramskij.html"
#dummy_url_again = "http://memory.pvost.org/pages/abrih.html"
dummy_url_again = "http://memory.pvost.org/pages/meletinskij.html"
page_east = urllib.request.urlopen(dummy_url_again)
soup_bio = BeautifulSoup(page_east)

In [54]:
east_bio = {}
info = soup_bio.find("p", {"class": "zag2"})
#biography = soup_bio.find("p", attrs={'class': None})
#biography = re.sub(r"\n\s{2,30}", "", biography.text)
#birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography.split("Род. в ")[1]).group()
    #if biography == 0:
    #    biography = soup_bio("p", attrs={'class': None})
    #    for item in biography:
    #        birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", item.text.split("Род. в ")[1]).group()
    #        if birthplace:
    #            east_bio["birthplace"] = birthplace
names = info.text
names = re.sub(r'\([^()]*\)', "", names)
names = re.sub(r'\([^()]*\)', "", names)
#names = re.sub(r'\(.*\)(.*\))?', "", names)
dates = re.search(r"((\d{4}\??|\?)-?(\d{4}\??|\?)?)", info.text).group()
surname = re.search(r"[А-ЯЁ]+-?[А-ЯЁ]+", names).group()
east_bio["surname"] = surname.title()
firstname = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]{0,10})|([А-ЯЁ]\.)", names).group()
east_bio["firstname"] = firstname.title()
birth = re.search(r"(\d{4}\??|\?)", dates).group()

try:
    patronym = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]+)", names.split(firstname+" ")[1]).group()
    east_bio["patronym"] = patronym
except:
    east_bio["patronym"] = ""
#    east_bio["birth_year"] = re.search(r"1[8-9][0-9]{2}", info.text.split("-")[0]).group()

try:
    east_bio["fullname"] = surname.title()+" "+firstname+" "+patronym
except:
    east_bio["fullname"] = surname.title()+" "+firstname
if re.search("(а|я|и)$", firstname) and firstname not in ambig_male:
    east_bio["gender"] = "женщина"
elif firstname in ambig_fem:
    east_bio["gender"] = "женщина"
else:
    east_bio["gender"] = "мужчина"
try:
    east_bio["birthdate"] = birth
except:
    east_bio["birthdate"] = ""
try:
    east_bio["death_date"] = re.search(r"1[8-9][0-9]{2}", dates.split(birth)[1]).group()
except:
    east_bio["death_date"] = ""
    
#try:
#    biography = soup_bio.find("p", attrs={'class': None})
#    biography = re.sub(r"\n\s{2,30}", "", biography.text)
#    birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography.split("Род. в ")[1]).group()
#    if birthplace:
#        east_bio["birthplace"] = birthplace
#    else:
#        biography = soup_bio("p", attrs={'class': None})
#        print(biography)
#        for item in biography:
#            birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", item.text.split("Род. в ")[1]).group()
#            if birthplace:
#                east_bio["birthplace"] = birthplace
#except:
#    east_bio["birthplace"] = ""  
#try: 
#    east_bio["birthplace"] = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography.split("Род. в ")[1]).group()
#except:
#    east_bio["birthplace"] = ""
biography = soup_bio("p", attrs={'class': None})
#print(biography)
for i in range(len(biography)):
    birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography[1].text.split("Род. в ")[1]).group()
    if birthplace:
        east_bio["birthplace"] = birthplace
print(east_bio)
print(names)


{'surname': 'Мелетинский', 'firstname': 'Елеазар', 'patronym': 'Моисеевич', 'fullname': 'Мелетинский Елеазар Моисеевич', 'gender': 'мужчина', 'birthdate': '1918', 'death_date': '', 'birthplace': 'г. Харьков'}
МЕЛЕТИНСКИЙ 
                      Елеазар 
                      Моисеевич 
                      


In [60]:
east_bio = {}
info = soup_bio.find("p", {"class": "zag2"})
#biography = soup_bio.find("p", attrs={'class': None})
#biography = re.sub(r"\n\s{2,30}", "", biography.text)
#birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography.split("Род. в ")[1]).group()
    #if biography == 0:
    #    biography = soup_bio("p", attrs={'class': None})
    #    for item in biography:
    #        birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", item.text.split("Род. в ")[1]).group()
    #        if birthplace:
    #            east_bio["birthplace"] = birthplace
names = info.text
names = re.sub(r'\([^()]*\)', "", names)
names = re.sub(r'\([^()]*\)', "", names)
#names = re.sub(r'\(.*\)(.*\))?', "", names)
dates = re.search(r"((\d{4}\??|\?)-?(\d{4}\??|\?)?)", info.text).group()
surname = re.search(r"[А-ЯЁ]+-?[А-ЯЁ]+", names).group()
east_bio["surname"] = surname.title()
firstname = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]{0,10})|([А-ЯЁ]\.)", names).group()
east_bio["firstname"] = firstname.title()
birth = re.search(r"(\d{4}\??|\?)", dates).group()

try:
    patronym = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]+)", names.split(firstname+" ")[1]).group()
    east_bio["patronym"] = patronym
except:
    east_bio["patronym"] = ""
#    east_bio["birth_year"] = re.search(r"1[8-9][0-9]{2}", info.text.split("-")[0]).group()

try:
    east_bio["fullname"] = surname.title()+" "+firstname+" "+patronym
except:
    east_bio["fullname"] = surname.title()+" "+firstname
if re.search("(а|я|и)$", firstname) and firstname not in ambig_male:
    east_bio["gender"] = "женщина"
elif firstname in ambig_fem:
    east_bio["gender"] = "женщина"
else:
    east_bio["gender"] = "мужчина"
try:
    east_bio["birthdate"] = birth
except:
    east_bio["birthdate"] = ""
try:
    east_bio["death_date"] = re.search(r"1[8-9][0-9]{2}", dates.split(birth)[1]).group()
except:
    east_bio["death_date"] = ""
try: 
    biography = soup_bio("p", attrs={'class': None})
    for i in range(len(biography)):
        birth_check = re.search(r"[а-яё]+", biography[0].text)
        if birth_check:
            biography_info = re.sub(r"\n\s{2,30}", "", biography[0].text)
            birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography_info.split("Род. в ")[1]).group()
            if birthplace:
                east_bio["birthplace"] = birthplace
        else:
            birth_check = re.search(r"[а-яё]+", biography[1].text)
            if birth_check:
                biography_info = re.sub(r"\n\s{2,30}", "", biography[1].text)
                birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography_info.split("Род. в ")[1]).group()
                if birthplace:
                    east_bio["birthplace"] = birthplace
            else:
                biography_info = re.sub(r"\n\s{2,30}", "", biography[2].text)
                birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography_info.split("Род. в ")[1]).group()
                if birthplace:
                    east_bio["birthplace"] = birthplace
                
#except (AttributeError, IndexError):
#    biography = soup_bio.find("p", attrs={'class': None})
#    biography = re.sub(r"\n\s{2,30}", "", biography.text)
#    east_bio["birthplace"] = re.search(r"[А-ЯЁ]{0,2}[а-яё]{1,7}\.?[А-Яа-яЁё\s-]+(?=\.|,|;)", biography.split("Род. в ")[1]).group()
    
#print(type(biography.text))
    #if type(biography.text) is str:
    #    biography = re.sub(r"\n\s{2,30}", "", biography.text)
    #    east_bio["birthplace"] = re.search(r"[А-ЯЁ]{0,2}[а-яё]{1,7}\.?[А-Яа-яЁё\s-]+(?=\.|,|;)", biography.split("Род. в ")[1]).group()
    #else:
        #biography = soup_bio("p", attrs={'class': None})
     #   for i in range(len(biography)):
     #       birthplace = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography[1].text.split("Род. в ")[1]).group()
      #      if birthplace:
      #          east_bio["birthplace"] = birthplace
except:
    east_bio["birthplace"] = ""
print(east_bio)
print(names)

{'surname': 'Мелетинский', 'firstname': 'Елеазар', 'patronym': 'Моисеевич', 'fullname': 'Мелетинский Елеазар Моисеевич', 'gender': 'мужчина', 'birthdate': '1918', 'death_date': '', 'birthplace': 'г. Харьков'}
МЕЛЕТИНСКИЙ 
                      Елеазар 
                      Моисеевич 
                      


In [51]:
with open('vostokovedy4_base.json', 'w', encoding='utf-8') as f:
    json.dump(biographies, f, ensure_ascii=False, indent=4)

In [50]:
biographies

[{'surname': 'Абих',
  'firstname': 'Рудольф',
  'patronym': 'Петрович',
  'fullname': 'Абих Рудольф Петрович',
  'gender': 'мужчина',
  'birthdate': '1901',
  'birthplace': 'г. Баку',
  'death_date': '1940',
  'original_link': 'http://memory.pvost.org/pages/abrih.html',
  'retrieved_at': '2021-05-18'},
 {'surname': 'Абрамский',
  'firstname': 'Иехезкель',
  'patronym': '',
  'fullname': 'Абрамский Иехезкель',
  'gender': 'мужчина',
  'birthdate': '1886',
  'birthplace': 'м. Дашковичи Гродненской губ',
  'death_date': '1976',
  'original_link': 'http://memory.pvost.org/pages/abramskij.html',
  'retrieved_at': '2021-05-18'},
 {'surname': 'Абрамсон',
  'firstname': 'Мануил',
  'patronym': 'Моисеевич',
  'fullname': 'Абрамсон Мануил Моисеевич',
  'gender': 'мужчина',
  'birthdate': '1898',
  'birthplace': 'др. свед',
  'death_date': '1938',
  'original_link': 'http://memory.pvost.org/pages/abramson.html',
  'retrieved_at': '2021-05-18'},
 {'surname': 'Абугов',
  'firstname': 'Михаил',
  '